**1. Installs**

In [1]:
'''! pip install openai
! pip install pinecone-client
! pip install langchain[extras]
! pip install -U langchain-community
! pip install tiktoken
'''

'! pip install openai\n! pip install pinecone-client\n! pip install langchain[extras]\n! pip install -U langchain-community\n! pip install tiktoken\n'

In [2]:
import gradio as gr

**2.Setup pinecone**

In [3]:
import pinecone
import json
import os
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="alejandra2"
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENV")  # Región de Pinecone

print("Pinecone instalado y funcionando correctamente.")

Pinecone instalado y funcionando correctamente.


In [4]:
from pinecone import Pinecone, ServerlessSpec

# Crear una instancia de Pinecone con tu API Key
pc = Pinecone(api_key=pinecone_api_key)

# Listar índices disponibles
existing_indexes = pc.list_indexes()
print("Índices existentes:", [i["name"] for i in existing_indexes])

Índices existentes: ['alejandra-project', 'alejandra-project2', 'about-conditions-index', 'alejandra-combined']


In [5]:
# Especificar la región y proveedor (serverless)
spec = ServerlessSpec(cloud="aws", region=pinecone_env)

index_name = "alejandra-project2"
# Conectar al índice
index = pc.Index(index_name)
print(f"Índice '{index_name}' conectado.")

Índice 'alejandra-project2' conectado.


**3.Setup Retriever**

In [32]:
from langchain.vectorstores import Pinecone

from langchain.embeddings.openai import OpenAIEmbeddings

# Configurar OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


# Configurar el vectorstore con Pinecone
vectorstore = Pinecone.from_existing_index(
    index_name="alejandra-project2",  # Nombre del índice creado en Pinecone
    embedding=embeddings,           # Embeddings de OpenAI (ya configurados)
    text_key="question"             # Campo en los metadatos donde se almacena la pregunta
)

response = "¿puedes bucar informacion sobre alejandra roncancio y sus terapias en los documentos dados?"
# Perform similarity search
vectorstore.similarity_search(
    response,  # Our search query
    k=3  # Return 3 most relevant docs
)

'''# Convertir el vectorstore en un retriever
retriever = vectorstore.as_retriever()'''

print("✅ Contexto inicial añadido al agente.")


✅ Contexto inicial añadido al agente.


----extra cell embedding-----

In [21]:
# Realizar una consulta de prueba
query = "¿Qué pasa en una terapia psicológica?"
query_embedding = embeddings.embed_query(query)

# Buscar en Pinecone usando argumentos con nombre
results = index.query(vector=query_embedding, top_k=3, include_metadata=True)



In [22]:
faq_retriever = Pinecone.from_existing_index(
    index_name="alejandra-project2",  # Nombre correcto del índice de FAQ
    embedding=embeddings,
    text_key="answer"  # Clave del campo que contiene las preguntas
).as_retriever()

**Updating prompt**

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [33]:
system_prompt = (
    "Utiliza el contexto dado para responder a la pregunta."
    "Si no sabes la respuesta, di que no la sabes."
    "Utiliza un máximo de tres frases y sé conciso en la respuesta."
    "El nombre de la persona dueña de la informacion que te he dado es Alejandra Roncancio y ofrece terapias a clientes de habla hispana y alemana en Alemania."
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Alejandra Roncancio"
}
```
Observation: No sé quién es Alejandra Roncancio.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "No tengo información específica sobre Alejandra Roncancio. Para asegurarte de que estás en manos de un profesional, puedes verificar sus credenciales, buscar reseñas de otros pacientes y confirmar que esté registrada en una asociación profesional de psicología. También es importante que te sientas cómodo y seguro durante las sesiones."
}
```

> Finished chain.


**4. Conversational Agent with langchain**

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory


# Configurar el modelo de lenguaje (ChatOpenAI)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # Modelo de OpenAI
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0  # Respuestas más determinísticas
)

# Configurar el QA Chain con el retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faq_retriever,
    chain_type="stuff",  # Recupera contexto y genera una respuesta
)

# Configurar herramientas para el agente
tools = [
    Tool(
        name="Knowledge Base",
        func=qa_chain.run,
        description="Usa esta herramienta para responder preguntas sobre psicología y terapias basadas en el conocimiento disponible y ademas informacion sobre Alejandra Roncancio."
    )
]


# Configurar memoria conversacional
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # Clave para almacenar el historial de la conversación
    return_messages=True,       # Devuelve mensajes completos
    k=3                         # Recuerda hasta 5 mensajes previos
)

# Inicializar el agente conversacional
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="chat-conversational-react-description",  # Tipo de agente
    verbose=True,
    memory=memory,
    early_stopping_method='generate'
)


/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_25371/1251242280.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_25371/1251242280.py:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_25371/1251242280.py:44: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for buildin

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

**TEST**

In [26]:
# Probar el agente con una pregunta inicial
response = agent.run("¿Qué pasa en una terapia psicológica?")
print("Respuesta del agente:", response)

/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_25371/1807255936.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("¿Qué pasa en una terapia psicológica?")




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "¿Qué pasa en una terapia psicológica?"
}
```
Observation: En una terapia psicológica, el proceso puede variar dependiendo del enfoque del terapeuta y las necesidades del paciente. Generalmente, en la primera cita, el terapeuta se dedica a conocer al paciente, hacer preguntas y escuchar las razones que lo llevan a la consulta. Con esta información, el terapeuta puede determinar la forma de trabajo más adecuada y transmitir sus impresiones sobre cómo abordará el tratamiento.

En las citas posteriores, se pueden trabajar los resultados de tareas propuestas, realizar ejercicios, o preparar al paciente para técnicas específicas, como el EMDR en casos de procesamiento de trauma. El enfoque se centra en lo que trae al paciente a la consulta y en los aspectos de su vida relacionados con sus dificultades.

Es importante mencionar que si una persona presenta síntomas muy incapacitantes o se enc

In [30]:
response = agent.run("¿Informacion de contacto?")
print("Respuesta del chatbot (General Info):", response)




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Información de contacto de Alejandra Roncancio"
}
```
Observation: No tengo información sobre Alejandra Roncancio.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Lo siento, no tengo información de contacto sobre Alejandra Roncancio."
}
```

> Finished chain.
Respuesta del chatbot (General Info): Lo siento, no tengo información de contacto sobre Alejandra Roncancio.




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "¿Alejandra Roncancio es certificada en EMDR?"
}
```
Observation: No sé si Alejandra Roncancio es certificada en EMDR.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "No tengo información sobre si Alejandra Roncancio es certificada en EMDR."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Es natural tener dudas sobre la autenticidad de una persona o servicio. Para asegurarte de que no es un engaño, puedes investigar sobre la persona o entidad en cuestión, buscar reseñas o testimonios de otros clientes, y verificar su formación y credenciales. También puedes preguntar directamente sobre sus métodos y experiencia. La transparencia y la comunicación abierta son buenas señales de confianza."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Ba

In [28]:
response = agent.run("¿cuanto cuesta una cita o consulta?")
print("Respuesta del chatbot (General Info):", response)




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "cuanto cuesta una cita o consulta"
}
```
Observation: Los costos de las citas o consultas son los siguientes:

- Primera cita o consulta individual: 60€ por 60 minutos.
- Primera cita o consulta de pareja o familiar: 80€ por 60 minutos.
- Citas o consultas posteriores individuales: 80€ por 60 minutos.
- Citas o consultas posteriores de pareja o familiares: 130€ por 90 minutos.
- Cita o consulta individual de EMDR: 100€ por 90 minutos.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Los costos de las citas o consultas son los siguientes:\n\n- Primera cita o consulta individual: 60€ por 60 minutos.\n- Primera cita o consulta de pareja o familiar: 80€ por 60 minutos.\n- Citas o consultas posteriores individuales: 80€ por 60 minutos.\n- Citas o consultas posteriores de pareja o familiares: 130€ por 90 minutos.\n- Cita o consulta individual de EMDR: 100€ por 90 minutos

gradio

In [46]:
import gradio as gr

# Función para conectar el chatbot con Gradio
def agente_chatbot(user_input):
    try:
        response = agent.run(user_input)  # Llamamos al agente para generar la respuesta
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Configurar la interfaz de Gradio con estilos personalizados
with gr.Blocks(css="""
    @import url('https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;600&display=swap');
    
    .gradio-container {
        background-color: #f5f5f5;
        font-family: 'Open Sans', sans-serif !important;
    }

    button { 
        background-color: #d50001 !important; 
        color: white !important; 
        border-radius: 5px !important;
        font-family: 'Open Sans', sans-serif !important;
        font-weight: 600 !important;
    }
""") as demo:
    gr.Markdown("<h1 style='text-align: center; color: #000000; font-family: Open Sans;'>Chatbot de Alejandra Roncancio</h1>")
    gr.Markdown("<p style='text-align: center; font-family: Open Sans;'>Chatea con un agente basado en LangChain y Pinecone.</p>")
    
    with gr.Row():
        user_input = gr.Textbox(label="Pregúntame lo que quieras", placeholder="Escribe aquí tu pregunta...", scale=8)
        submit_button = gr.Button("Enviar", variant="primary", scale=1)

    output = gr.Textbox(label="Respuesta", interactive=False)

    submit_button.click(fn=agente_chatbot, inputs=user_input, outputs=output)

# Lanzar la aplicación de Gradio
demo.launch(pwa=True,share=True)



* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://3b12b609999064971e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "que es trauma"
}
```
Observation: No sé.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "El trauma se refiere a una experiencia emocional o psicológica que resulta de un evento extremadamente estresante o perturbador. Puede incluir experiencias como abuso, accidentes, desastres naturales, o la pérdida de un ser querido. El trauma puede afectar la salud mental y emocional de una persona, y puede manifestarse a través de síntomas como ansiedad, depresión, flashbacks y dificultades en las relaciones interpersonales."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "cuanto cuesta una consulta"
}
```
Observation: La primera cita o consulta individual cuesta 60€ por 60 minutos. Las citas o consultas posteriores individuales cuestan 80€ por 60 minutos. Si se trata de una consulta de pareja o f